# POC of model finetuning using AISG FAQ data
Note: this is not a rigorous proof of the finetuning abilities, it merely shows that we are able to overfit to one dataset. The main purpose is to build and test the finetuning code using tensorflow.  
Tested with just minimizing cosine loss. This kind of works, but having a triplet loss might be better.  
Triplet takes many more epochs before it gets the same accuracy as just minimizing cosine loss, but I think it should be more robust. Will have to try both on the training dataset.

In [1]:
import sys
sys.path.append('..')
import numpy as np
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
from src.utils import read_txt, split_txt, aiap_qna_quickscore
import tensorflow as tf
import tensorflow_hub as hub

/anaconda/envs/py35/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/anaconda/envs/py35/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/anaconda/envs/py35/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/anaconda/envs/py35/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
from src.model import GoldenRetriever
model=GoldenRetriever(loss='triplet')

Instructions for updating:
Colocations handled automatically by placer.


W1106 07:07:14.476074 140634671175424 deprecation.py:323] From /anaconda/envs/py35/lib/python3.5/site-packages/tensorflow/python/ops/control_flow_ops.py:3632: colocate_with (from tensorflow.python.framework.ops) is deprecated and will be removed in a future version.
Instructions for updating:
Colocations handled automatically by placer.


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


I1106 07:07:29.406564 140634671175424 saver.py:1483] Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


...

Instructions for updating:
Use tf.cast instead.


W1106 07:07:44.159590 140634671175424 deprecation.py:323] From /anaconda/envs/py35/lib/python3.5/site-packages/tensorflow/python/ops/losses/losses_impl.py:331: to_float (from tensorflow.python.ops.math_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.cast instead.


Instructions for updating:
Use tf.cast instead.


W1106 07:07:44.298229 140634671175424 deprecation.py:323] From /anaconda/envs/py35/lib/python3.5/site-packages/tensorflow/python/ops/math_ops.py:3066: to_int32 (from tensorflow.python.ops.math_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.cast instead.


model initiated!


In [3]:
aiap_qa, aiap_context = split_txt(read_txt('../data/aiap.txt'), qa=True)

In [4]:
print(aiap_context)

['Q1. WHAT SORT OF CANDIDATES ARE YOU LOOKING FOR?', 'Q2. WHAT WILL BE COVERED IN THE PROGRAMME?', 'Q3. DO I HAVE TO PAY?', 'Q4. WHAT IS THE OUTCOME OF THIS PROGRAMME?', 'Q5. WILL I GET A JOB AFTER THE PROGRAMME?', 'Q6. WILL THERE BE A NEXT ROUND OF APPLICATION?', 'Q7. DO I HAVE TO GIVE UP MY CURRENT JOB TO JOIN THE  PROGRAMME?', 'Q8. WHAT IF I DECIDED TO DROP OUT OF THE PROGRAMME?', 'Q9. WILL I GET SOME SORT OF CERTIFICATE TO SHOW THAT I PARTICIPATED IN THE AIAP?', 'Q10. WHAT ARE THE AI JOBS AND ROLES I CAN CONSIDER AFTER COMPLETING THE TRAINING PROGRAMME?']


In [5]:
model.predict(aiap_qa, type='query')

array([[ 0.0112743 ,  0.04894138, -0.06490619, ...,  0.09330797,
         0.01958819, -0.02537908],
       [ 0.04272488,  0.0623336 ,  0.02158852, ...,  0.06216078,
         0.00950298, -0.03111232],
       [-0.05566482,  0.02913201,  0.0255424 , ...,  0.0459285 ,
         0.00670882, -0.07619023],
       ...,
       [-0.01923553, -0.06181138,  0.01505679, ...,  0.07817681,
         0.02878742, -0.07518731],
       [-0.014769  , -0.05544962,  0.09171879, ...,  0.0126055 ,
        -0.02055403, -0.05686105],
       [ 0.05459761,  0.03379358,  0.01586364, ...,  0.03275432,
         0.04136325,  0.03180375]], dtype=float32)

In [6]:
for k in range(3):
    print(k, aiap_qna_quickscore(aiap_context, model.predict(aiap_qa, type='response'), aiap_qa, model, k+1))

0 0.5
1 0.6
2 0.6


# Load finetuned

In [8]:
model.restore('/data/home/lik/info_retrieve/google_use_qa_tuned/variables-0')

INFO:tensorflow:Restoring parameters from /data/home/lik/info_retrieve/google_use_qa_tuned/variables-0


I1106 07:08:42.459116 140634671175424 saver.py:1270] Restoring parameters from /data/home/lik/info_retrieve/google_use_qa_tuned/variables-0


model checkpoint restored!


In [9]:
for k in range(5):
    print(k, aiap_qna_quickscore(aiap_context, model.predict(aiap_qa, type='response'), aiap_qa, model, k+1))

0 0.8
1 1.0
2 1.0
3 1.0
4 1.0


# Finetune model
overfitting on default questions will happen between 10-15 epoch on cosine loss. Model checkpointed with 15 epoch of training with shuffled mistakes.

In [7]:
import random
aiap_context_trim=[' '.join(x.split(' ')[1:]) for x in aiap_context]
aiap_context_trim

def shuffle_mistakes():
    aiap_ans_neg = aiap_qa.copy()
    shuffle_ind = [ii for ii in range(10)]
    random.shuffle(shuffle_ind)
    aiap_ans_neg=[aiap_ans_neg[ii] for ii in shuffle_ind]
    return aiap_ans_neg
aiap_ans_neg = shuffle_mistakes()

In [13]:
for _ in range(5):
    aiap_ans_neg = shuffle_mistakes()
    for answer, question, neg_qn in zip(aiap_qa, aiap_context_trim, aiap_ans_neg):
#         print(answer, question)
        print(model.finetune([question], [answer], [answer], [neg_qn], [neg_qn]))

0.0
0.0
0.015765548
0.29999995
0.3
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.23655546
0.30000007
0.0
0.0
0.0
0.0
0.3
0.0
0.0
0.3
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.3
0.0
0.0
0.088243246
0.0
0.0
0.0
0.0
0.06774145
0.0
0.0
0.00056535006
0.0
0.0


In [14]:
for k in range(3):
    print(k, aiap_qna_quickscore(aiap_context_trim, model.predict(aiap_qa), aiap_qa, model, k+1))

0 1.0
1 1.0
2 1.0


In [ ]:
# model.export(savepath='../google_use_qa_tuned/variables')

In [13]:
model.close()

In [ ]:
model=QnaEncoderModel(loss='cosine')

5 epoch 0.7

# Fine tuning on toy dataset works! Now to try for insurance dataset
This is more rigorous, cause I will be testing on unseen data.
## First load the test framework for qna


In [3]:
from src.utils import question_cleaner, display_qn_and_ans, aiap_qna, ranker, scorer
from pathlib import Path
import random
import pickle

In [4]:
datapath=Path('../data')
df_query = pd.read_csv(datapath/'insuranceQA/V2/InsuranceQA.question.anslabel.raw.500.pool.solr.train.encoded', delimiter='\t', header=None)
df_doc = pd.read_csv(datapath/'insuranceQA/V2/InsuranceQA.label2answer.raw.encoded', delimiter='\t', header=None)
df_ind2word = pd.read_csv(datapath/'insuranceQA/V2/vocabulary', sep='\t', header=None, quotechar='', quoting=3, keep_default_na=False)
dict_ind2word = pd.Series(df_ind2word[1].values,index=df_ind2word[0].values).to_dict()

df_test = pd.read_csv(datapath/'insuranceQA/V2/InsuranceQA.question.anslabel.raw.100.pool.solr.test.encoded', delimiter='\t', header=None)
df_test=question_cleaner(df_test)


total:2000, removed:677, remainder:1323


In [5]:
df_query=question_cleaner(df_query)
df_doc=df_doc.set_index(0)

def func(row):
    kb=[int(xx) for xx in (row[3]).split(' ')]
    gt = [int(xx) for xx in (row[2]).split(' ')]
    return random.sample([xx for xx in kb if xx not in gt], 100)
df_query['neg_samples']=df_query.apply(lambda x: func(x), axis=1)

total:12889, removed:2498, remainder:10391


In [6]:
def wordifier(tokes):
    return ' '.join([dict_ind2word[ind] for ind in tokes.strip().split(' ')])
df_doc['text']=df_doc.apply(lambda x: wordifier(x[1]), axis=1)
df_query['text']=df_query.apply(lambda x: wordifier(x[1]), axis=1)
df_test['text']=df_test.apply(lambda x: wordifier(x[1]), axis=1)
display(df_query.head(1))
display(df_test.head(1))
display(df_doc.head(1))

,0,1,2,3,neg_samples,text
1,life-insurance,idx_3019 idx_16371 idx_5499 idx_448 idx_136 id...,26354,13324 23395 7870 15080 27181 25052 26694 568 2...,"[127, 21311, 22635, 11029, 4920, 2125, 6970, 1...",Can Creditors Take Life Insurance After Death?


,0,1,2,3,text
4,medicare-insurance,idx_2363 idx_467 idx_8080 idx_31 idx_9966 idx_...,9128,9128 13322 21601 21471 6442 5412 24861 23536 2...,Will Medicare Pay For Smoking Cessation?


,1,text
0,,
1,idx_1 idx_2 idx_3 idx_4 idx_5 idx_6 idx_7 idx_...,Coverage follows the car. Example 1: If you we...


In [7]:
def get_hard_wrong(question, all_wrong_ans, model, k=1):
    wrong_answer_array=model.predict(all_wrong_ans, type='response')
    sorted_ans, sortargs, similarity_score=aiap_qna(question, wrong_answer_array, all_wrong_ans, model, k=50)
    return sorted_ans[0]

def generate_triplets(df_query):
    answers=[]
    questions=[]
    wrong_answers=[]
    for ii, row in df_query.iterrows():
        all_correct_ans = [int(xx) for xx in row[2].split(' ')]
        all_wrong_ans = df_doc.loc[row['neg_samples'], 'text'].tolist()
        for ans in all_correct_ans:
            answers.append(df_doc.loc[ans, 'text'])
            questions.append(row['text'])
            wrong_answers.append(get_hard_wrong(row['text'], all_wrong_ans, model))
    return answers, questions, wrong_answers


In [8]:
# answers, questions, wrong_answers=generate_triplets(df_query)
# with open(datapath/"tmp_trainset.pickle", "wb") as f:
#     pickle.dump((answers,questions,wrong_answers), f)


In [9]:
with open(datapath/"tmp_trainset.pickle", "rb") as f:
    answers,questions,wrong_answers = pickle.load(f) 

In [10]:
random.shuffle(wrong_answers)

In [10]:
batch_size=100
for _ in range(1):
    for ii in range(0, len(answers), batch_size):
        (kb, query, neg_query) = (questions[ii:ii+batch_size], answers[ii:ii+batch_size], wrong_answers[ii:ii+batch_size])
        current_loss = model.finetune(kb, query, query, neg_query)
        print(ii, current_loss)
            

0 0.024368823
100 0.0042471886
200 0.010198474
300 0.0
400 0.0
500 0.0
600 0.0
700 0.0
800 0.012136817
900 0.013827622
1000 0.0
1100 0.0052496195
1200 0.0049781203
1300 0.0025287867
1400 0.0
1500 0.0
1600 0.0
1700 0.0
1800 0.0
1900 0.0
2000 0.0
2100 0.0
2200 0.0
2300 0.0
2400 0.0
2500 0.001412332
2600 0.0
2700 0.0
2800 0.0
2900 0.0
3000 0.0
3100 0.0
3200 0.0009291172
3300 0.0
3400 0.0
3500 0.0
3600 0.0
3700 0.0
3800 0.0
3900 0.0
4000 0.0
4100 0.0
4200 0.0
4300 0.0
4400 0.0
4500 0.0
4600 0.0
4700 0.0
4800 0.0
4900 0.0
5000 0.0
5100 0.0
5200 0.0
5300 0.0
5400 0.0
5500 0.0
5600 0.0
5700 0.0
5800 0.0
5900 0.0
6000 0.0
6100 0.0
6200 0.0
6300 0.0
6400 0.0
6500 0.0
6600 0.0
6700 0.0
6800 0.0
6900 0.0
7000 0.0
7100 0.0
7200 0.0
7300 0.0
7400 0.0
7500 0.0
7600 0.0
7700 0.0
7800 0.0
7900 0.0
8000 0.0
8100 0.0
8200 0.0
8300 0.0
8400 0.0
8500 0.0
8600 0.0
8700 0.0
8800 0.0
8900 0.0
9000 0.0
9100 0.0
9200 0.0
9300 0.0
9400 0.0
9500 0.0
9600 0.0
9700 0.0
9800 0.0
9900 0.0
10000 0.0
10100 0.0
10200 0

In [11]:
question_vectors = model.predict(df_test['text'].tolist(), type='query')
print('questions vectorized!')
predictions, gts = ranker(model, question_vectors, df_test, df_doc)
for k in range(5):
    print('Score @{}: {:.4f}'.format(k+1, scorer(predictions, gts, k+1)))

questions vectorized!
Score @1: 0.3915
Score @2: 0.5200
Score @3: 0.5949
Score @4: 0.6493
Score @5: 0.6984


In [11]:
for k in range(5):
    print('Score @{}: {:.4f}'.format(k+1, scorer(predictions, gts, k+1)))

Score @1: 0.3870
Score @2: 0.5193
Score @3: 0.5896
Score @4: 0.6478
Score @5: 0.6977


## Pretrained
Score @1: 0.3870  
Score @2: 0.5193  
Score @3: 0.5896  
Score @4: 0.6478  
Score @5: 0.6977  

## Using cosine loss (1 epoch, lr=0.1)
Score @1: 0.4067  
Score @2: 0.5427  
Score @3: 0.6153  
Score @4: 0.6667  
Score @5: 0.7067  

## Using triplet loss (1 epoch, lr=0.1, margin=0.1) no shuffling of data
Score @1: 0.3915  
Score @2: 0.5200  
Score @3: 0.5949  
Score @4: 0.6493  
Score @5: 0.6984  